In [8]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime


if torch.cuda.is_available():
    device = torch.device("cuda")
    print("You are using GPU")
else:
    print("You are using CPU")

You are using GPU


In [9]:
df = pd.read_csv('data_processing/landmark.csv')
class ProcessData(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.iloc[:, 1:].values
        self.labels = dataframe.iloc[:, 0].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {
            'data': torch.tensor(self.data[idx], dtype=torch.float32, device=device),
            'label': torch.tensor(self.labels[idx], dtype=torch.long, device=device)
        }
        return sample

keypoint_dataset = ProcessData(df)
print(keypoint_dataset.__len__())

4998


In [10]:
import model as m
model = m.SimpleNN2().to(device)
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

dataloader = DataLoader(keypoint_dataset, batch_size = 128, shuffle = True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Model's state_dict:
fc1.weight 	 torch.Size([40, 63])
fc1.bias 	 torch.Size([40])
fc2.weight 	 torch.Size([20, 40])
fc2.bias 	 torch.Size([20])
fc3.weight 	 torch.Size([10, 20])
fc3.bias 	 torch.Size([10])
fc4.weight 	 torch.Size([5, 10])
fc4.bias 	 torch.Size([5])


In [11]:
num_epochs = 200
previous_loss = float('inf')  # Initialize with a high value

for epoch in range(num_epochs):
    for batch in dataloader:
        inputs = batch['data']
        labels = batch['label']

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    if loss < previous_loss:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        previous_loss = loss  # Update the previous loss

torch.save(model.state_dict(), "models/model2.pth")

Epoch [1/200], Loss: 1.6010
Epoch [2/200], Loss: 1.4189
Epoch [4/200], Loss: 1.2597
Epoch [6/200], Loss: 1.1958
Epoch [7/200], Loss: 0.9266
Epoch [21/200], Loss: 0.9060
Epoch [46/200], Loss: 0.9056
Epoch [71/200], Loss: 0.9050
Epoch [85/200], Loss: 0.9049
Epoch [108/200], Loss: 0.9048
Epoch [117/200], Loss: 0.9048
Epoch [130/200], Loss: 0.9048
